## IMPORTS ##

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Imports
! pip install calplot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import plotly.express as px
import pandas as pd
import calplot
import seaborn as sns
import geopandas as gpd



import warnings
warnings.filterwarnings("ignore")

dep_lvl_edu = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/depression_by_level_of_educatio.csv')
num_dep_count = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/number_with_depression_by_count.csv')
prev_mental_subst = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_by_mental_and_substa.csv')
prev_dep_age =  pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_of_depression_by_age.csv')
prev_dep_males =  pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_of_depression_males.csv')
suicide_vs_prev =  pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/suicide_rates_vs_prevalence_of.csv')

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for calplot: filename=calplot-0.1.7.5-py3-none-any.whl size=8117 sha256=f86ad57433158249dc2a541096e3ca9a49fe68a34ef71074f20de853e723a363
  Stored in directory: /root/.cache/pip/wheels/77/75/32/a518a4a1421776a93e1ede33cb8dac474095bf31fed7e4e22c
Successfully built calplot


In [2]:
dep_lvl_edu = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/depression_by_level_of_educatio.csv')
num_dep_count = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/number_with_depression_by_count.csv')
prev_mental_subst = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_by_mental_and_substa.csv')
prev_dep_age =  pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_of_depression_by_age.csv')
prev_dep_males =  pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_of_depression_males.csv')
suicide_vs_prev =  pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/suicide_rates_vs_prevalence_of.csv')

## 1. MOTIVATION ##

The datasets we chose are datasets that represents a wide collection of different disorders and mental health and statistics from the population across many countries.

In specific the datasets we used to tell our story were:
- Prevalence by Mental and Substance Abuse
- Prevalence of depression by Age
- Prevalence of Depression by Gender
- Suicide Rates vs Prevalence of Depression



We chose the datasets because mental health problems has become very common worldwide recently, and it seems like it is an increasing trend. We wanted to furhter investigate this problem, and see if we could gain any insights as to why mental health problems are on the rise in the recent years.


We wanted to tell a story for the end user on how depression around the world looks like, and what the potential causes and correlations to depression could be. The end user then hopefully after reading our story gains some insights on the state of mental health problems, and some reasonings behind said problems.

## 2. BASIC STATS ##

We only did very preliminary data processing as the data we aqcuired was very clean to begin with. The most crucial data preprocessing we did was remove any rows with missing values. Another small edit as well was renaming a country to make it work with the cloropleth map. For the depression prevalence dataset across ages we had to correct some of the columns as the age gaps were not properly written.

Four datasets have been studied in this project:

1.	Prevalence of Mental Disorders
This dataset provides data on the prevalence of various mental disorders (schizophrenia, bipolar disorder, anxiety, eating disorders, drug use, alcohol use, and depressive disorders) by country over the years. The dataset size is 807 KB and 6469 rows.
2.	Prevalence of Depression by Gender
This dataset contains data on the prevalence of depression by country, gender, and across multiple years. The dataset size is 1.6 MB and 47808 rows.
3.	Prevalence of Depression by Age
This dataset includes information on the prevalence of depression by country, age group, and over several years. Age groups covered are "10-14," "15-19," "20-24," "25-29," "30-34," "35-49," "50-69," and "70+," with data also available for "All ages." The dataset size is 1.1 MB and 6469 rows.
4.	Suicide Rate vs Depression Rate
This dataset features information on suicide rates per 100,000 individuals and the prevalence of depression per 100,000 individuals, by country and over the years. The dataset size is 1.6 MB 47808.

All four datasets were found from the following source:  https://ourworldindata.org/mental-health#research-writing


The geo data was used to show the prevalence of depression by each country, as can be seen in Plot 1. Plot 2 also shows the aggrevated data from all the countries to show the correlations between the different disorders. Plot 3 shows the some pairs of the disorders, their relation and their temporal trend.

Plots:

In [11]:
#@title Plot 1
import plotly.graph_objs as go
colorbar=dict(title="Prevalence (%)"),  # Color bar title


def plot_world_map_with_slider(df, column_name):
    fig = go.Figure()

    # Iterate through each year
    for year in range(1990, 2017):
        filtered_df = df[df['year'] == year]  # Filter data for the current year
        trace = go.Choropleth(
            locations=filtered_df['entity'],  # Location data
            z=filtered_df[column_name],  # Data to be visualized
            locationmode='country names',  # Location mode
            colorscale='OrRd',  # Color scale
                colorbar=dict(title="Prevalence (%)"),  # Corrected color bar title

            zmin=df[column_name].min(),  # Min value of the data
            zmax=df[column_name].max(),  # Max value of the data
            visible=False  # Initially set trace to not visible
        )
        fig.add_trace(trace)  # Add the trace to the figure

    fig.data[0].visible = True  # Set the first trace to visible

    steps = []
    for i in range(len(fig.data)):
        step = dict(
            method='update',
            args=[{'visible': [False] * len(fig.data)},  # Set all traces to invisible
                  {'title_text': f'{column_name} Map - {1990 + i}', 'frame': {'duration': 1000, 'redraw': True}}],
            label=str(1990 + i)
        )
        step['args'][0]['visible'][i] = True  # Set the current trace to visible
        steps.append(step)

    sliders = [dict(
        active=0,
        steps=steps,
        currentvalue={"prefix": "Year: ", "font": {"size": 14}},
    )]

    fig.update_layout(
        title_text='Depression Prevalence Worldwide (1990-2017)',
        title_font_size=24,
        title_x=0.5,
        geo=dict(
            showframe=True,
            showcoastlines=True,
            projection_type='natural earth'
        ),
        sliders=sliders,
        height=500,
        width=1000,
        font=dict(family='Arial', size=12),
        margin=dict(t=80, l=50, r=50, b=50),
    )

    # Save the plot to an HTML file
    return fig

# Example usage
map = plot_world_map_with_slider(prev_mental_subst, 'depression')
map

In [10]:
#@title Plot 2
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, ColorBar, LinearColorMapper
from bokeh.transform import linear_cmap
from bokeh.palettes import RdGy11  # Gray-red color palette
import numpy as np
import math  # For using math.pi

# Load data
data_issues2 = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_by_mental_and_substa.csv')
data_issues2 = data_issues2[data_issues2['year'] == 2017]

# Calculate correlation
data_issues_corr_gl = data_issues2[['schizophrenia', 'bipolar_disorder', 'eating_disorders', 'anxiety_disorders', 'drug_use_disorders', 'depression', 'alcohol_use_disorders']]
corrs = data_issues_corr_gl.corr()

# Prepare the data for the heatmap
source = ColumnDataSource(data={
    'x': np.repeat(list(corrs.columns), len(corrs.index)),
    'y': np.tile(list(corrs.index), len(corrs.columns)),
    'z': corrs.values.flatten(),
    'text': corrs.round(2).values.flatten().astype(str)
})

# Create a Bokeh figure
p = figure(
    title='Correlation Heatmap of Disorders in Greenland',
    x_range=list(corrs.columns),
    y_range=list(corrs.index)[::-1],  # Reverse y-axis to match heatmap order
    tools='hover,pan,box_zoom,reset',
    width=800,
    height=800
)

# Define color mapper using the gray-red palette
color_mapper = LinearColorMapper(palette=RdGy11, low=-1, high=1)

# Add the heatmap using rect glyphs
p.rect(
    x='x', y='y', width=1, height=1,
    source=source,
    fill_color=linear_cmap('z', palette=RdGy11, low=-1, high=1),
    line_color=None
)

# Add a hover tool
hover = HoverTool()
hover.tooltips = [
    ('Correlation', '@text')
]
p.add_tools(hover)

# Add a color bar
color_bar = ColorBar(color_mapper=color_mapper, location=(0, 0))
p.add_layout(color_bar, 'right')

# Set x-axis labels at a 45-degree angle
p.xaxis.major_label_orientation = math.pi / 4  # 45 degrees in radians
p.xaxis.major_label_text_font_size = "10pt"

# Display the plot
output_notebook()
show(p)


In [13]:
#@title Plot 3
import pandas as pd
import plotly.graph_objects as go

# Load your CSV data
data_issues = pd.read_csv('/content/drive/MyDrive/vizzup-mental-health-depression-disorder-data/data/prevalence_by_mental_and_substa.csv')
country = "Greenland"
data_issues = data_issues[data_issues['entity'] == country]

# Prepare plot data for multiple years
def prepare_data(column):
    return data_issues[['entity', 'year', 'depression', column]].dropna()

# Prepare data
data_anxiety = prepare_data('anxiety_disorders')
data_drug_use = prepare_data('drug_use_disorders')
data_alcohol_use = prepare_data('alcohol_use_disorders')

# Create plot
fig = go.Figure()

# Function to add scatter plot to the figure
def add_scatter(name, data, x_col, y_col, title):
    fig.add_trace(
        go.Scatter(
            x=data[x_col],  # This now represents the disorder
            y=data[y_col],  # This now represents depression
            mode='markers',
            name=name,
            visible=False,
            marker=dict(
                size=8,
                color=data['year'],  # Use year for color coding
                colorbar=dict(title='Year'),
                colorscale='Reds'  # Using Reds colorscale for a more reddish hue
            ),
            text=data['entity']  # Show entity name on hover
        )
    )

# Add plots with initial titles for the x-axis and y-axis
add_scatter('Anxiety vs Depression', data_anxiety, 'anxiety_disorders', 'depression', "Prevalence of Depression against Anxiety Disorders in Greenland (1990-2017)")
add_scatter('Drug Use vs Depression', data_drug_use, 'drug_use_disorders', 'depression', "Prevalence of Depression against Drug Use Disorders in Greenland (1990-2017)")
add_scatter('Alcohol Use vs Depression', data_alcohol_use, 'alcohol_use_disorders', 'depression', "Prevalence of Depression against Alcohol Use Disorders in Greenland (1990-2017)")

# Update layout for the dropdown
fig.update_layout(
    width=800,  # Set width to make the plot wider
    height=500,  # Set height to increase the plot height
    updatemenus=[
        {
            "buttons": [
                {"method": "update", "label": "Anxiety vs Depression",
                 "args": [{"visible": [True, False, False]},
                          {"xaxis": {"title": "Anxiety Disorders Prevalence (%)"},
                           "yaxis": {"title": "Depression (%)"},
                           "title": "Prevalence of Depression against Anxiety Disorders in Greenland (1990-2017)"}]},
                {"method": "update", "label": "Drug Use vs Depression",
                 "args": [{"visible": [False, True, False]},
                          {"xaxis": {"title": "Drug Use Disorders  Prevalence (%)"},
                           "yaxis": {"title": "Depression  Prevalence (%)"},
                           "title": "Prevalence of Depression against Drug Use Disorders in Greenland (1990-2017)"}]},
                {"method": "update", "label": "Alcohol Use vs Depression",
                 "args": [{"visible": [False, False, True]},
                          {"xaxis": {"title": "Alcohol Use Disorders  Prevalence (%)"},
                           "yaxis": {"title": "Depression  Prevalence (%)"},
                           "title": "Prevalence of Depression against Alcohol Use Disorders in Greenland (1990-2017)"}]}
            ],
            "direction": "down",
            "pad": {"r": 10, "t": 10},
            "showactive": True,
            "x": 1.05,  # Position on the x-axis to move the menu to the right
            "xanchor": "left",
            "y": 1.14,  # Position on the y-axis to move the menu to a lower position
            "yanchor": "top"
        }
    ],
    title="Scatter plot of Depression against other Mental Disorders in Greenland (1990-2017)",  # Initial title
    xaxis_title="Mental Disorder Prevalence (%)",
    yaxis_title="Depression Prevalence (%)",
    coloraxis_colorbar=dict(
        yanchor="top",
        y=1.07  # Adjust position to move the color bar above the plot title
    )
)

# Show first plot by default and set the initial y-axis label
fig.data[0].visible = True

fig

## 3. DATA ANALYSIS ##

The data analysis we conducted through the study was aimed at learning about mental health disorders. The keyfindings and insights we learned from the datasets were:
- Depression Prevalence in Greenland - The analysis shows that Greenland consistently showed to have the highest rates of depression prevalance compared to the global average. Another point is that the prevalance of depression of Greenland does have a slow downwards trend but however is still very high compared to the rest of the world.

- Correlation with Other Disorders - Through the use of a correlation map we also learned that there is a correlation between depression and other disorders such as anxiety and drug use disorders, indicating that these disorders may be connected.

- Greenland vs Europe vs Worldwide - In the same trend as the prevalance of depression in Greenland, through the analysis we also see that Greenland also has a significant higher rate in alcohol, anxiety, and drug use disorders compared to both Europe and the rest of the world.

- Depression in ages and gender - Our analysis also showed that in Greenland that more females tend to be suffering from depression disorder. Furthermore, through a bar chart we also saw that the young age group from 20-24 in Greenland also are more prone to depression disorder.

- Trend of multivariate analysis - Through the scatterplot we also learned that in fact the prevelance of depression is on a downwards decline but however anxiety and drug use disorders are on the rise. We speculated that this was due to the main focus on depression, while the other disorders being neglected leading to their rise. This is also different from the global trends and also leads to show some specific Greenland dynamics that could be the reason behind this.

## 4. GENRE ##

We told our story using the magazine style from Segel and Heer [1], as we wanted to tell a story with both clarifying text and visualizations, and the magazine style is therefore the best option.

The tools from the visual narrative categories from Segel and Heer we used for our story were:
- Feature Distinction - We used feature distinction by highlighting Greenland in comparison to other countries.

- Zooming - Zooming was implemented for the Geo Map so the end user is able to zoom and explore the map with greater detail

The tools from the narrative structure categories from Segel and Heer we used for our story were:
- Introdoctory text - We have introdoctory text to introduce the end user to the topic and give some background information for the story.

- Linear - The story is linear as we start from finding the country with most prevalent depression, and explore the reasons and delve deeper into Greenland.

- Hover highlighting - Most of the visualizations also have hover highlighting allowing the end user to get the details of the visualizations.

- Filtering - We used the filtering when using the scatter plot to save space in the story and also allow the end user to explore the filtered data that they find interesting.

## 5. VISUALIZATIONS ##

These were the types of visualizations we used: Line chart, Bar Chart, Geo Map, Correlation Map, and Scatter Plot.

- Line Chart - We chose the line charts because we wanted to show the development of different disorders with a temporal aspect. The line chart also allows us to very easily compare the difference between two lines, allowing us to explore the data in an easy manner.

- Bar Chart - The bar charts allowed the end user to compare the age groups in an easy manner and combined with the Distinction tool from Segal and Heer makes it very easy and clear for the end user to spot the age group with most prevelance of depression.

- Geo map - The map visualization we chose to give the end user a bigger overview of how the prevelance of depression is all over the world but also to highlight the almost stark difference between Greenland and the rest of the world.

- Correlation Map - The correlation map allows the end user to visually see the correlations the different disorders have with each other. This was used for the story as the end user can gain many insights on how the disorders affect each other, and it allows the end user to also see other correlations than the ones we talked about in our story

- Scatter plot - The scatter plot in combination with the  narrative structure techniques Hover Highlithing and Filtering allows the end user to explore the development of any pair of the three disorders: Anxiety, Depression and Alcohol versus Depression. The end user can filter each pair and also get more details by hovering over each point. Furthermore, the coloring of the dots also allows the end user to see the general temporal trend, gaining even more insights on each of the pairs of disorders.

We chose these visualizations as most of our key insights have a temporal aspect, or use some kind of comparison be it either between age groups, gender or other countries. The visualization types we picked allowed us to tell the story with those things in mind.

## 6. DISCUSSION ##

The exploration of these datasets revealed intriguing trends and correlations in depression prevalence and related factors when studying Greenland's data alone and comparing it to other countries.

While some may argue that interactive plots are not useful in cases where there are no prominent changes over time, such as depression prevalence, they still serve to highlight the persistence of certain issues across time.

Potential limitations include the exclusion of other valuable data, such as additional socioeconomic factors or distributions of factors associated with depression, such as drug use disorders, among the population. Additionally, it would have been interesting to observe the evolution of different variables over time based on their geographical location, as certain regions of Greenland have been shown to report a higher prevalence of depression.

## 7. CONTRIBUTIONS ##

|                    | Thomas | Melis | Jesus |
|--------------------|--------|-------|-------|
| Visualizations     | 20%    | 40%   | 40%   |
| Website            | 20%    | 30%   | 50%   |
| Part A             | 33%    | 33%   | 33%   |
| Explainer Notebook | 100%   | 0%    | 0%    |

## 8. REFERENCES ##


[1] E. Segel and J. Heer, "Narrative Visualization: Telling Stories with Data," in IEEE Transactions on Visualization and Computer Graphics, vol. 16, no. 6, pp. 1139-1148, Nov.-Dec. 2010, doi: 10.1109/TVCG.2010.179.
Keywords: {Data visualization;Visualization;Media;Image color analysis;Engineering profession;Economics;Narrative visualization;storytelling;design methods;case study;journalism;social data analysis}

